In [1]:
## This is demo for kuka reaching a with mpc and diff_qp
## Author : Avadesh Meduri
## Date : 25/02/2022

import time
import numpy as np
import pinocchio as pin
from robot_properties_kuka.config import IiwaConfig

import meshcat
import meshcat.transformations as tf
import meshcat.geometry as g

from diff_pin_costs import DiffFrameTranslationCost
from inverse_qp import IOC

import torch
from torch.autograd import Function
from torch.nn import functional as F

In [2]:
robot = IiwaConfig.buildRobotWrapper()
model, data = robot.model, robot.data
f_id = model.getFrameId("EE")

/opt/openrobots/lib/python3.8/site-packages/pinocchio/shortcuts.py:45: UserWarning: You passed package dir(s) via argument geometry_model and provided package_dirs.
  geom_model = pin.buildGeomFromUrdf(model, filename, geometry_type, package_dirs)


In [3]:
viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=False)
viz.loadViewerModel()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [5]:
class DiffFramePlacementCost(Function):
    """
    This cost provides gradients wrt joint position and velocity for the final end effector/frame positions
    """
    @staticmethod
    def forward(ctx, state, model, data, f_id, des_frame):
        """
        Input:
            state : vector (q, dq)
            model : pinocchio robot model
            data : pinocchio robot data
            f_id : frame id for which derivatives are desired
            des_frame : desired frame
        """
        state = state.double().detach().numpy()
        nq, nv = model.nq, model.nv
        pin.forwardKinematics(model, data, state[0:nq], state[nq:nq + nv], np.zeros(nv))
        pin.updateFramePlacements(model, data)
        
        inv_des_frame = pin.SE3.inverse(pin.SE3(des_frame))
        SE3_error = inv_des_frame * self.data.oMf[self.f_id]
        se3_error = pin.log6(SE3_error)

        J = pin.computeFrameJacobian(model, data, state[0:nq], f_id, pin.ReferenceFrame.LOCAL)       
        
        # derivative wrt q
        dq = torch.tensor(np.matmul(np.matmul(pin.Jlog6(SE3_error), J).T,se3_error))
        der = torch.hstack((dq, torch.zeros(nv)))
        ctx.der = dq
    
        return torch.tensor(0.5*np.linalg.norm(se3_error)**2)

    @staticmethod
    def backward(ctx, grad):
        
        der = ctx.der
         # derivative wrt joint positions
        
        return der@grad, None, None, None
        

In [6]:
dtc = DiffFrameTranslationCost.apply

def quadratic_loss(q_pred, x_des, nq, n_col):
    loss = 1e2*torch.linalg.norm(dtc(q_pred[-2*nq:], model, data, f_id) - x_des)
    loss += 5*torch.linalg.norm(q_pred[-nq:])
    for i in range(n_col):    
        loss += 0.8*torch.linalg.norm(dtc(q_pred[(3*i)*nq: (3*i+2)*nq], model, data, f_id) - x_des)
        loss += 1e-3*torch.linalg.norm(q_pred[(3*i+2)*nq: (3*i+3)*nq]) # control regularization
        loss += 1e-1*torch.linalg.norm(q_pred[(3*i+1)*nq: (3*i+2)*nq]) # velocity regularization
        loss += 1e-1*torch.linalg.norm(q_pred[(3*i)*nq: (3*i+1)*nq]) # joint regularization
    
    return loss

In [7]:
dfc = DiffFramePlacementCost.apply

def frame_loss(q_pred, x_des, nq, n_col):
    tmp = np.eye(4)
    tmp[:,3][0:3] = x_des
    
    loss = 1e2*dfc(q_pred[-2*nq:], model, data, f_id, frame_des)
    loss += 5*torch.linalg.norm(q_pred[-nq:])
    for i in range(n_col):    
        loss += 0.8*torch.linalg.norm(dtc(q_pred[(3*i)*nq: (3*i+2)*nq], model, data, f_id) - x_des)
        loss += 1e-3*torch.linalg.norm(q_pred[(3*i+2)*nq: (3*i+3)*nq]) # control regularization
        loss += 1e-1*torch.linalg.norm(q_pred[(3*i+1)*nq: (3*i+2)*nq]) # velocity regularization
        loss += 1e-1*torch.linalg.norm(q_pred[(3*i)*nq: (3*i+1)*nq]) # joint regularization
    
    return loss

In [13]:
nq = model.nq
nv = model.nv

n_col = 5
u_max = [2.5,2.5,2.5, 1.5, 1.5, 1.5, 1.0]

lr = 1e-1
eps = 80

# q_des = np.hstack(((np.pi/4)*(np.random.randint(0, 2, size = 5)), np.zeros(2)))

q_des_arr = np.array([[ 0.3009,  1.1532,  1.7729,  1.7383,  1.2195, -0.0204,  0.0593]])
# dq_des = np.zeros_like(q_des)
# pin.forwardKinematics(model, data, q_des, dq_des, np.zeros(nv))
# pin.updateFramePlacements(model, data)

x_des_arr = torch.tensor([[0.5, -0.4, 0.4]]) #[0.6, 0.4, 1.0]

# q_init = np.hstack(((np.pi/8)*np.random.choice(list(range(-2, -1)) + list(range(1, 3)), size=(4)), np.zeros(3)))

x_init = np.zeros(2*nq)
x_init[0:nq] = q_des_arr[0] + 0.5*np.random.rand(nq)
# x_init[0:nq] = np.hstack(((np.pi/12)*(np.random.randint(3, 6, size = 5)), np.zeros(2)))
x_init[nq:] = 0.2*2*(np.random.rand(nv) - 0.5)



In [14]:
x_in = x_init
x_des = x_des_arr[np.random.randint(len(x_des_arr))]

viz.viewer["box"].set_object(g.Sphere(0.05), 
                         g.MeshLambertMaterial(
                             color=0xff22dd,
                             reflectivity=0.8))
viz.viewer["box"].set_transform(tf.translation_matrix(x_des.detach().numpy()))


for j in range(20):

    ioc = IOC(n_col, nq, u_max, 0.05, eps = 1.0, isvec=True)
    optimizer = torch.optim.Adam(ioc.parameters(), lr=lr)
    
    
    i = 0
    loss = 1000.
    old_loss = 10000.
    
    while loss > 0.03 and i < eps and abs(old_loss - loss) > 5e-5:
        x_pred = ioc(x_in) 

        old_loss = loss

        loss = quadratic_loss(x_pred, x_des, nq, n_col)
        print("Index :" + str(i) + " loss is : " + str(loss.detach().numpy()), end = '\r', flush = True)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1
        
    x_pred = ioc(x_in).detach().numpy()

    for i in range(n_col+1):
        q = x_pred[3*nq*i:3*nq*i + nq]
        dq = x_pred[3*nq*i + nq:3*nq*i + 2*nq]

        pin.forwardKinematics(model, data, q, dq, np.zeros(nv))
        pin.updateFramePlacements(model, data)

        viz.display(q)
        time.sleep(0.1)
    
    x_in = torch.tensor(x_pred[-2*nq:])
    print(x_in[-2*nq:-nq])

tensor([0.4673, 1.4628, 1.9255, 2.0358, 1.6508, 0.0877, 0.2659],
       dtype=torch.float64)
tensor([0.6289, 1.3494, 1.8331, 1.9620, 1.6266, 0.1276, 0.2648],
       dtype=torch.float64)
tensor([0.8843, 1.2140, 1.7169, 1.8320, 1.5945, 0.1787, 0.2642],
       dtype=torch.float64)
tensor([1.2334, 1.1099, 1.5977, 1.6431, 1.5484, 0.2316, 0.2621],
       dtype=torch.float64)
tensor([1.5013, 1.0322, 1.5155, 1.4096, 1.5059, 0.2229, 0.2599],
       dtype=torch.float64)
tensor([1.6130, 0.9615, 1.5142, 1.2523, 1.5266, 0.1962, 0.2586],
       dtype=torch.float64)
tensor([1.5808, 0.9138, 1.5532, 1.1887, 1.5526, 0.1708, 0.2578],
       dtype=torch.float64)
tensor([1.4550, 0.9493, 1.5944, 1.2157, 1.5784, 0.1451, 0.2571],
       dtype=torch.float64)
tensor([1.3737, 0.9711, 1.6139, 1.2188, 1.5669, 0.1236, 0.2565],
       dtype=torch.float64)


KeyboardInterrupt: 

In [ ]:
torch.utils.cmake_prefix_path

In [ ]:
x_train_loaded = torch.load("./data/x_train3.pt")
y_train_loaded = torch.load("./data/y_train3.pt")

In [ ]:
# i = 50
# n_vars = 3*nq*n_col + 2*nq
# data = y_train_loaded[i]
# Q = torch.reshape(data[0:n_vars**2], (n_vars, n_vars))
# q = torch.nn.Parameter(data[n_vars**2:])
# print(q[-2*nq:-nq])
# print(q[-nq:])
# print(Q[-nq:,-nq:]@q[-nq:])